In [83]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [84]:
path_train="../No_window/nw_scores/nw_stack_train/"
path_test="../No_window/nw_scores/nw_stack_test/"

In [3]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [4]:
col_no_ID=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7',
       'class8', 'class9']

In [5]:
#Affecting  columns name
for name in dict_test.keys():
    dict_train[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]
    dict_test[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]

In [6]:
stack_train=dict_train["nw_xgb_d2v"].merge(dict_train["nw_xgb_tfidf"],
on="ID").merge(dict_train["nw_xgb_w2v"],on="ID").merge(dict_train["nw_lgbm_d2v"],
on="ID").merge(dict_train["nw_lgbm_tfidf"],on="ID").merge(dict_train["nw_lgbm_w2v"],
on="ID").merge(dict_train["nw_lr_d2v"],on="ID").merge(dict_train["nw_lr_tfidf"],
on="ID").merge(dict_train["nw_lr_w2v"],on="ID").merge(dict_train["nw_ada_d2v"],
on="ID").merge(dict_train["nw_ada_tfidf"],on="ID").merge(dict_train["nw_ada_w2v"],
on="ID").merge(dict_train["nw_rf_d2v"],on="ID").merge(dict_train["nw_rf_tfidf"],
on="ID").merge(dict_train["nw_rf_w2v"],on="ID")
#.merge(dict_train["nowdw_neural_net"],on="ID")
#a tester 

In [7]:
stack_test=dict_test["nw_xgb_d2v"].merge(dict_test["nw_xgb_tfidf"],
on="ID").merge(dict_test["nw_xgb_w2v"],on="ID").merge(dict_test["nw_lgbm_d2v"],
on="ID").merge(dict_test["nw_lgbm_tfidf"],on="ID").merge(dict_test["nw_lgbm_w2v"],
on="ID").merge(dict_test["nw_lr_d2v"],on="ID").merge(dict_test["nw_lr_tfidf"],
on="ID").merge(dict_test["nw_lr_w2v"],on="ID").merge(dict_test["nw_ada_d2v"],
on="ID").merge(dict_test["nw_ada_tfidf"],on="ID").merge(dict_test["nw_ada_w2v"],
on="ID").merge(dict_test["nw_rf_d2v"],on="ID").merge(dict_test["nw_rf_tfidf"],
on="ID").merge(dict_test["nw_rf_w2v"],on="ID")

In [8]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [28]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [43]:
ID_test=stack_test["ID"]
ID_train=stack_train["ID"]

In [52]:
meta_feat_train=pd.read_csv("../No_window/nw_meta_features/meta_train_l1l2.csv")
meta_feat_test=pd.read_csv("../No_window/nw_meta_features/meta_test_l1l2.csv")

In [54]:
meta_feat_train=meta_feat_train.drop("Unnamed: 0",axis=1)
meta_feat_test=meta_feat_test.drop("Unnamed: 0",axis=1)

In [55]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [56]:
stack_train_1=pd.concat((stack_train,meta_feat_train,magic_train),axis=1)
stack_test_1=pd.concat((stack_test,meta_feat_test,magic_test),axis=1)

In [61]:
X_train=np.array(stack_train_1.drop("ID",axis=1))

In [62]:
X_test=np.array(stack_test_1.drop("ID",axis=1))

In [85]:
#ALL molecular here

In [88]:
magic_feat_all=pd.read_csv("../l2_meta_features/all_molecular_functions.csv")
magic_feat_all=magic_feat_all.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train_all=magic_feat_all[:len(y)]
magic_test_all=magic_feat_all[len(y):]
magic_train_all=magic_train_all.reset_index(drop=True)
magic_test_all=magic_test_all.reset_index(drop=True)
stack_train_1_all=pd.concat((stack_train,meta_feat_train,magic_train_all),axis=1)
stack_test_1_all=pd.concat((stack_test,meta_feat_test,magic_test_all),axis=1)
X_train_all=np.array(stack_train_1_all.drop("ID",axis=1))
X_test_all=np.array(stack_test_1_all.drop("ID",axis=1))

In [93]:
stack_train_1_all

,nw_xgb_d2v_class1,nw_xgb_d2v_class2,nw_xgb_d2v_class3,nw_xgb_d2v_class4,nw_xgb_d2v_class5,nw_xgb_d2v_class6,nw_xgb_d2v_class7,nw_xgb_d2v_class8,nw_xgb_d2v_class9,ID,...,epidermal growth factor receptor binding [GO:0005154],phospholipase binding [GO:0043274],patched binding [GO:0005113],integrin binding [GO:0005178],hepatocyte growth factor-activated receptor activity [GO:0005008],transcription cofactor binding [GO:0001221],core promoter sequence-specific DNA binding [GO:0001046],platelet-derived growth factor binding [GO:0048407],"phosphatidylinositol-3,4-bisphosphate binding [GO:0043325]",interleukin-12 receptor binding [GO:0005143]
0,0.927897,0.010089,0.004072,0.029766,0.002735,0.009338,0.015073,0.000582,0.000448,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.028777,0.032007,0.028167,0.841315,0.031233,0.015568,0.013519,0.008012,0.001403,1.0,...,1,0,0,0,0,0,0,0,0,0
2,0.040167,0.095287,0.020159,0.663344,0.067525,0.064521,0.026755,0.019106,0.003136,2.0,...,1,0,0,0,0,0,0,0,0,0
3,0.017270,0.039240,0.011321,0.824411,0.062000,0.027887,0.011942,0.004235,0.001694,3.0,...,1,0,0,0,0,0,0,0,0,0
4,0.023391,0.030937,0.038434,0.743194,0.092233,0.043833,0.018403,0.007724,0.001851,4.0,...,1,0,0,0,0,0,0,0,0,0
5,0.021271,0.022479,0.052001,0.827262,0.024861,0.025398,0.017926,0.007490,0.001313,5.0,...,1,0,0,0,0,0,0,0,0,0
6,0.017484,0.048547,0.016145,0.828829,0.039540,0.024566,0.011028,0.012665,0.001197,6.0,...,1,0,0,0,0,0,0,0,0,0
7,0.318400,0.031876,0.009073,0.472011,0.007670,0.112659,0.022496,0.023806,0.002009,7.0,...,1,0,0,0,0,0,0,0,0,0
8,0.055651,0.020523,0.009551,0.646383,0.038687,0.172401,0.040504,0.013772,0.002529,8.0,...,1,0,0,0,0,0,0,0,0,0
9,0.043361,0.182572,0.051524,0.539783,0.121252,0.031764,0.022898,0.004373,0.002474,9.0,...,1,0,0,0,0,0,0,0,0,0


In [89]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.1,0.3,0.5,0.7,0.9, 1, 2,3,4,5],
    "penalty": ["l1","l2"]
}


In [90]:
kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [91]:
gsearch_all=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch_all.fit(X_train_all,y)
gsearch_all.grid_scores_, gsearch_all.best_params_, gsearch_all.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.92860, std: 0.03846, params: {'C': 0.1, 'penalty': 'l1'},
  mean: -0.93270, std: 0.04035, params: {'C': 0.1, 'penalty': 'l2'},
  mean: -0.89180, std: 0.04057, params: {'C': 0.3, 'penalty': 'l1'},
  mean: -0.92479, std: 0.03962, params: {'C': 0.3, 'penalty': 'l2'},
  mean: -0.88700, std: 0.04379, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.91973, std: 0.04234, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.88459, std: 0.04633, params: {'C': 0.7, 'penalty': 'l1'},
  mean: -0.91713, std: 0.04482, params: {'C': 0.7, 'penalty': 'l2'},
  mean: -0.88711, std: 0.04898, params: {'C': 0.9, 'penalty': 'l1'},
  mean: -0.91698, std: 0.04472, params: {'C': 0.9, 'penalty': 'l2'},
  mean: -0.88831, std: 0.04859, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.91661, std: 0.04494, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.89210, std: 0.06072, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.91425, std: 0.04525, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.90074, std: 0.06744, params: {'

In [75]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.92888, std: 0.04135, params: {'C': 0.1, 'penalty': 'l1'},
  mean: -0.93251, std: 0.04187, params: {'C': 0.1, 'penalty': 'l2'},
  mean: -0.89225, std: 0.04256, params: {'C': 0.3, 'penalty': 'l1'},
  mean: -0.92617, std: 0.04280, params: {'C': 0.3, 'penalty': 'l2'},
  mean: -0.88581, std: 0.04346, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.92481, std: 0.04126, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.88120, std: 0.04637, params: {'C': 0.7, 'penalty': 'l1'},
  mean: -0.92502, std: 0.04321, params: {'C': 0.7, 'penalty': 'l2'},
  mean: -0.87942, std: 0.04865, params: {'C': 0.9, 'penalty': 'l1'},
  mean: -0.92403, std: 0.04246, params: {'C': 0.9, 'penalty': 'l2'},
  mean: -0.87965, std: 0.04914, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.92379, std: 0.04452, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.88506, std: 0.05983, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.92370, std: 0.04491, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.88828, std: 0.06379, params: {'

In [ ]:
#model with parameters found on gridsearch

In [78]:
def stacker_gen(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/nw_stack_test.csv",index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    

    y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("../final_scores/nw_stack_train.csv",index=False)


In [79]:
# paramètre C et penalty à rentrer

In [82]:
clf = LogisticRegression(C=0.9,penalty="l1")
stacker_gen(X_train,X_test,y,clf)

Fold 1 (2946, 197) (743, 197)
Fold 2 (2949, 197) (740, 197)
Fold 3 (2952, 197) (737, 197)
Fold 4 (2954, 197) (735, 197)
Fold 5 (2955, 197) (734, 197)
cross_val sur train 
